# Valuation for Qlib Training

In [ ]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from libs import compute_ic, rebalance

## Load Results

We first read all the results that qlib training genereted

In [ ]:
exp_name = 'dens/oos'

# read result
path = Path(f'../data/intermediate/results/{exp_name}')
pred = pd.read_parquet(path)
pred = pred.loc[~pred.index.duplicated(keep='last')]

# read return data and benchmark data
ret = pd.read_parquet('../data/intermediate/forward_return/1d_open_open.parquet')
benchmark = pd.read_parquet('../data/index/zz500.parquet', columns=['ret'])
benchmark = benchmark.loc[pred.index.levels[0]]

# construct pred_label variable
pred_label = pd.concat([pred, ret], axis=1, join='inner')

## Rebalance by 1 Day

In [ ]:
result = rebalance(pred_label, benchmark=benchmark, N=10, commission_ratio=0.002)

### Layering Result

In [ ]:
(result['profit'] + 1).cumprod().plot(title='Cumulative Return', figsize=(12, 8));

In [ ]:
(result['exprofit'] + 1).cumprod().plot(title='Cumulative Execess Return', figsize=(12, 8));

In [ ]:
annual_ret = (np.power((
        result['profit'].iloc[:, -2] + 1).cumprod().iloc[-1], 252 / len(result['profit']
    )) - 1) * 100
annual_exret = (np.power((
        result['exprofit'].iloc[:, -1] + 1).cumprod().iloc[-1], 252 / len(result['profit']
    )) - 1) * 100
print(f"Top Layer Average Annual Return: {annual_ret:.2f}%")
print(f"Top Layer Average Annual Execess Return: {annual_exret:.2f}%")

### Draw Down

In [ ]:
result['mmd'].plot(title='Profit Draw Down', figsize=(12, 8));

In [ ]:
result['exmmd'].plot(title='Excess Profit Draw Down', figsize=(12, 8));

In [ ]:
mmd = result['mmd'].iloc[:, -2].min() * 100
exmmd = result['exmmd'].iloc[:, -1].min() * 100
print(f"Top Layer Max Draw Down: {mmd:.2f}%")
print(f"Top Layer Excess Max Draw Down: {exmmd:.2f}%")

## Turnover

In [ ]:
result['turnover'].plot(title='Turnover Rate', figsize=(12, 8));

## IC

In [ ]:
ic = compute_ic(pred_label)['ic']
ic.rolling(20).mean().plot(title='IC Rolling 20-day Mean', figsize=(12, 8));